### Current Speed

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime,date,timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import psycopg2
import pandas.io.sql as psql
from sqlalchemy import create_engine
import sshtunnel 
import pandas as pd
import psycopg2.extras
import csv

In [2]:
def get_scheduled_stoptimes():
    
    #SQL statement
    sql = """Select *
    from stop_lines_regularity
    order by trip_id, stop_sequence, arrival_time"""
    
    #database connection

    try:
        sshtunnel.SSH_TIMEOUT = 10.0
        with sshtunnel.SSHTunnelForwarder(
                ('3.83.88.154', 22),
                ssh_username='ec2-user',
                #CHANGE PATH TO rds-example.cer
                ssh_pkey='/Users/lauraforerocamacho/Downloads/rds-example.cer',
                remote_bind_address=('stib-mivb.cjo3cnpt9pci.us-east-1.rds.amazonaws.com', 5432)
        ) as tunnel:
                conn = psycopg2.connect(
                        host='127.0.0.1',
                        port=tunnel.local_bind_port,     
                        database="stibmivbdb",
                        user="postgres",
                        password="stib-mivb-db1")

                cur = conn.cursor()
                #perform sql and write result to dataframe
                cur.execute(sql)
                field_names = [i[0] for i in cur.description]
                query_results = pd.DataFrame(cur.fetchall(),columns=field_names)
                #close connection
                cur.close()
                
                return query_results
    except Exception as e:
            print(e)

In [3]:
df_squedule_times=get_scheduled_stoptimes()
df_squedule_times.head()

,route_id,service_id,trip_id,trip_headsign,direction_id,date_from,date_to,arrival_time,departure_time,real_date_from,...,route_type,start_date,end_date,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,2,236269500,112947202236269500,STOCKEL,0,2021-09-04,2021-09-18,07:30:56,07:30:56,2022-09-03,...,1,2021-09-04,2021-09-18,0,0,0,0,0,1,0
1,2,236269500,112947202236269500,STOCKEL,0,2021-09-04,2021-09-18,07:31:53,07:31:53,2022-09-03,...,1,2021-09-04,2021-09-18,0,0,0,0,0,1,0
2,2,236269500,112947202236269500,STOCKEL,0,2021-09-04,2021-09-18,07:33:33,07:33:33,2022-09-03,...,1,2021-09-04,2021-09-18,0,0,0,0,0,1,0
3,2,236269500,112947202236269500,STOCKEL,0,2021-09-04,2021-09-18,07:34:44,07:34:44,2022-09-03,...,1,2021-09-04,2021-09-18,0,0,0,0,0,1,0
4,2,236269500,112947202236269500,STOCKEL,0,2021-09-04,2021-09-18,07:36:08,07:36:08,2022-09-03,...,1,2021-09-04,2021-09-18,0,0,0,0,0,1,0


In [4]:
df_distances=pd.read_csv("./realability/distance_stops.csv")

In [5]:
#insert results to database table assessment_methods
def insert_into_db(df_methods):
    try:
        sshtunnel.SSH_TIMEOUT = 10.0
        with sshtunnel.SSHTunnelForwarder(
                ('3.83.88.154', 22),
                ssh_username='ec2-user',
                #CHANGE PATH TO rds-example.cer
                ssh_pkey='/Users/lauraforerocamacho/Downloads/rds-example.cer',
                remote_bind_address=('stib-mivb.cjo3cnpt9pci.us-east-1.rds.amazonaws.com', 5432)
        ) as tunnel:
                engine = create_engine('postgresql://{user}:{password}@{host}:{port}/{db}'.format(
                        host='127.0.0.1',
                        port=tunnel.local_bind_port,     
                        db="stibmivbdb",
                        user="postgres",
                        password="stib-mivb-db1" ))

                df_methods.to_sql('stop_lines_regularity_data', engine, if_exists='append',index=False)
    except Exception as e:
            print(e)

In [6]:
def get_speed(df_stop_times,df_distances):
    for i in range(0,len(df_stop_times)-1):
        print(i)
        c_stop=df_stop_times.loc[i].to_dict()
        s_stop=df_stop_times.loc[i+1].to_dict()
        tipc=c_stop["trip_id"]
        tips=s_stop["trip_id"]
        if(tipc!=tips ):
            continue
        if(tipc==tips):
            time_str_1 =c_stop["arrival_time"]
            time_str_2 =s_stop["arrival_time"]
            hour = time_str_1[0:2]
            if int(hour) > 23:
                #translate times over 24 hours (24:35 --> 00:35)
                time_str = time_str_1
                time_str_1 = time_str.replace(hour,str(0) + str(int(hour)-24))
            hour = time_str_2[0:2]
            if int(hour) > 23:
                #translate times over 24 hours (24:35 --> 00:35)
                time_str = time_str_2
                time_str_2 = time_str.replace(hour,str(0) + str(int(hour)-24))

            time_1=datetime.strptime(time_str_1, '%H:%M:%S')
            time_2=datetime.strptime(time_str_2, '%H:%M:%S')

            delta_time=(time_2-time_1).total_seconds()
            df_filter=df_distances[(df_distances.stop_1==c_stop["stop_id"])&(df_distances.stop_2==s_stop["stop_id"])]
            distances=df_filter[["distance_m"]].values
            distance=0
            if(len(distances)>0):
                distance=float(distances[0])
            speed=(float(distance/delta_time) if delta_time>0 else 0)
            result=[]
            result.append({ 'route_id':c_stop["route_id"],'trip_id':tipc,
            'stop_id_1':c_stop["stop_id"],'stop_id_2':s_stop["stop_id"],
            'delta_time':delta_time,'distance':distance,'speed':speed})

            df_methods = pd.DataFrame(result)
            insert_into_db(df_methods)
            print("Row inserted")    
    return []   


In [7]:
df_methods_speed=get_speed(df_squedule_times,df_distances)

0
Row inserted
1
Row inserted
2
Row inserted
3
Row inserted
4
Row inserted
5
Row inserted
6
Row inserted
7
Row inserted
8
Row inserted
9
Row inserted
10
Row inserted
11
Row inserted
12
Row inserted
13
Row inserted
14
Row inserted
15
Row inserted
16
Row inserted
17
Row inserted
18
Row inserted
19
Row inserted
20
21
Row inserted
22


2022-12-12 05:15:39,004| ERROR   | Socket exception: Connection reset by peer (54)


(psycopg2.OperationalError) SSL SYSCALL error: EOF detected

[SQL: select pg_catalog.version()]
(Background on this error at: https://sqlalche.me/e/14/e3q8)
Row inserted
23
Row inserted
24
Row inserted
25
Row inserted
26
Row inserted
27
Row inserted
28
Row inserted
29
Row inserted
30
Row inserted
31
Row inserted
32
Row inserted
33
Row inserted
34
Row inserted
35
Row inserted
36
Row inserted
37


2022-12-12 05:33:43,029| ERROR   | Socket exception: Connection reset by peer (54)
2022-12-12 05:33:43,329| ERROR   | Could not establish connection from local ('127.0.0.1', 49414) to remote ('stib-mivb.cjo3cnpt9pci.us-east-1.rds.amazonaws.com', 5432) side of the tunnel: open new channel ssh error: SSH session not active


(psycopg2.OperationalError) connection to server at "127.0.0.1", port 49414 failed: SSL SYSCALL error: Undefined error: 0
connection to server at "127.0.0.1", port 49414 failed: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.

(Background on this error at: https://sqlalche.me/e/14/e3q8)
Row inserted
38
Row inserted
39
Row inserted
40
Row inserted
41
42
Row inserted
43
Row inserted
44
Row inserted
45
Row inserted
46
Row inserted
47
Row inserted
48
Row inserted
49
Row inserted
50
Row inserted
51
Row inserted
52
Row inserted
53
Row inserted
54
Row inserted
55


...

Could not establish session to SSH gateway
Row inserted
93
Row inserted
94
Row inserted
95
Row inserted
96
Row inserted
97
Row inserted
98
Row inserted
99
Row inserted
100
Row inserted
101
Row inserted
102
Row inserted
103
Row inserted
104
105
Row inserted
106
Row inserted
107
Row inserted
108
Row inserted
109
Row inserted
110


2022-12-12 07:12:52,011| ERROR   | Could not open connection to gateway
2022-12-12 07:12:52,092| ERROR   | Could not connect to gateway 3.83.88.154:22 : Unable to connect to 3.83.88.154: [Errno 50] Network is down
2022-12-12 07:12:52,158| ERROR   | Could not connect to gateway 3.83.88.154:22 : Unable to connect to 3.83.88.154: [Errno 50] Network is down


Could not establish session to SSH gateway
Row inserted
111
Could not establish session to SSH gateway
Row inserted
112
Could not establish session to SSH gateway
Row inserted
113


2022-12-12 07:12:52,259| ERROR   | Could not connect to gateway 3.83.88.154:22 : Unable to connect to 3.83.88.154: [Errno 50] Network is down
2022-12-12 07:12:52,323| ERROR   | Could not connect to gateway 3.83.88.154:22 : Unable to connect to 3.83.88.154: [Errno 50] Network is down
2022-12-12 07:12:52,383| ERROR   | Could not connect to gateway 3.83.88.154:22 : Unable to connect to 3.83.88.154: [Errno 50] Network is down


Could not establish session to SSH gateway
Row inserted
114
Could not establish session to SSH gateway
Row inserted
115
Could not establish session to SSH gateway
Row inserted
116
Row inserted
117
Row inserted
118
Row inserted
119
Row inserted
120
Row inserted
121
Row inserted
122
Row inserted
123
Row inserted
124
Row inserted
125
126
Row inserted
127
Row inserted
128
Row inserted
129
Row inserted
130
Row inserted
131
Row inserted
132
Row inserted
133
Row inserted
134
Row inserted
135
Row inserted
136
Row inserted
137
Row inserted
138
Row inserted
139
Row inserted
140
Row inserted
141
Row inserted
142
Row inserted
143
Row inserted
144
Row inserted
145
Row inserted
146
147
Row inserted
148
Row inserted
149
Row inserted
150
Row inserted
151
Row inserted
152
Row inserted
153
Row inserted
154
Row inserted
155
Row inserted
156
Row inserted
157
Row inserted
158
Row inserted
159
Row inserted
160
Row inserted
161
Row inserted
162
Row inserted
163
Row inserted
164
Row inserted
165
Row inserted


### New Speed Calculation